# Segmenting and Clustering Neighborhoods in Toronto (Task Week 3)

## A) Create Dataframe

In [1]:
# Install libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [6]:
# Get data from Wikipedia using BeautifulSoup package
website_url = requests.get(wiki).text
soup = BeautifulSoup(website_url,'lxml')
wiki_link ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wiki_link).text
soup = BeautifulSoup(wiki_page,'xml')

In [37]:
#soup

In [12]:
# Create raw table as basis for pd.dataframe
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

for tr_cell in table.find_all('tr'):
    
    count = 1
    Postcode_x      = -1
    Borough_x       = -1
    Neighbourhood_x = -1
    
    for td_cell in tr_cell.find_all('td'):
        if count == 1: 
            Postcode_x = td_cell.text
        if count == 2: 
            Borough_x = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if count == 3: 
            Neighbourhood_x = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        count +=1
        
        if (Postcode_x == 'Not assigned' or Borough_x == 'Not assigned' or Neighbourhood_x == 'Not assigned'):
            
            continue
            
           
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            
            continue
            
            
    except:
        
        pass
    
    if(Postcode_x == -1 or Borough_x == -1 or Neighbourhood_x == -1):
        
        continue    
        
    Postcode.append(Postcode_x)
    Borough.append(Borough_x)
    Neighbourhood.append(Neighbourhood_x)

In [15]:
# Combine postal code areas with more than one neighbor
unique_p = set(Postcode)

Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

In [18]:
# Create pandas dataframe
toronto_dic = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dic)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M2H,North York,Hillcrest Village
1,M1E,Scarborough,"Morningside, West Hill"
2,M4W,Downtown Toronto,Rosedale
3,M5A,Downtown Toronto,Harbourfront
4,M6J,West Toronto,"Little Portugal, Trinity"


In [20]:
df_toronto.shape

(77, 3)

## B) Add geographical coordinates

In [26]:
geo_link ="https://cocl.us/Geospatial_data"
df_geo = pd.read_csv(geo_link)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [34]:
df_geo.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
df_comb = df_toronto.merge(df_geo, on='Postcode')
df_comb.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
2,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750


## C) Explore and cluster the neighborhoods in Toronto

In [38]:
# import addtional libraries (like in NY sample)
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [50]:
df_comb['Borough'].unique()

array(['North York', 'Scarborough', 'Downtown Toronto', 'West Toronto',
       'York', 'Etobicoke', 'East York', 'Central Toronto',
       'East Toronto'], dtype=object)

In [113]:
df_comb_t = df_comb.loc[df_comb['Borough'].isin(['Downtown Toronto', 'West Toronto','Central Toronto','East Toronto'])].reset_index(drop=True)
df_comb_t

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
2,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
3,M5S,Downtown Toronto,University of Toronto,43.662696,-79.400049
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M6K,West Toronto,"Exhibition Place, Parkdale Village",43.636847,-79.428191
6,M5L,Downtown Toronto,Commerce Court,43.648198,-79.379817
7,M6P,West Toronto,High Park,43.661608,-79.464763
8,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
9,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [114]:
# create map of Toronto using latitude and longitude values
latitude = 43.6532; longitude = -79.3832
map_toronto= folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_comb_t['Latitude'], df_comb_t['Longitude'], df_comb_t['Borough'], df_comb_t['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Map can be used to cluster and analyze Neighborhoods in Toronto

In [115]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'FZZH4G3YDDZI255F5MJSGZR5AX4IGE5PXJOAD1SHD3KZPN25' # your Foursquare ID
CLIENT_SECRET = '303WVNLKRKTRHCEK0P00T4G224RR3GYNKRMPJQKDDZA1NTWR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FZZH4G3YDDZI255F5MJSGZR5AX4IGE5PXJOAD1SHD3KZPN25
CLIENT_SECRET:303WVNLKRKTRHCEK0P00T4G224RR3GYNKRMPJQKDDZA1NTWR


In [116]:
#DGet data from Foursquare

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

In [131]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [133]:
toronto_venues = getNearbyVenues(names=df_comb_t['Neighbourhood'],
                                   latitudes=df_comb_t['Latitude'],
                                   longitudes=df_comb_t['Longitude']
                                  )

Rosedale
Harbourfront
Little Portugal, Trinity
University of Toronto
Berczy Park
Exhibition Place, Parkdale Village
Commerce Court
High Park
St. James Town
Lawrence Park
First Canadian Place, Underground city
Church and Wellesley
Design Exchange, Toronto Dominion Centre
Queen's Park
The Annex, Yorkville
Dovercourt Village
Runnymede, Swansea
Cabbagetown, St. James Town
The Beaches
India Bazaar
CN Tower, King and Spadina, Railway Lands, South Niagara
Deer Park, Rathnelly, South Hill
Parkdale, Roncesvalles
Moore Park
Riverdale
Forest Hill North
Chinatown, Grange Park, Kensington Market
Toronto Islands, Union Station


In [134]:
print(toronto_venues.shape)
toronto_venues.head()

(1174, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery


In [137]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"CN Tower, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",42,42,42,42,42,42
"Chinatown, Grange Park, Kensington Market",81,81,81,81,81,81
Church and Wellesley,83,83,83,83,83,83
Commerce Court,100,100,100,100,100,100
"Deer Park, Rathnelly, South Hill",14,14,14,14,14,14
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
Dovercourt Village,15,15,15,15,15,15


In [138]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 206 uniques categories.


In [139]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols=list(toronto_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
toronto_onehot=toronto_onehot[['Neighborhood']+cols]

# rename Neighborhood for Neighbourhood so that future merge works
toronto_onehot.rename(columns = {'Neighborhood': 'Neighbourhood'}, inplace = True)
toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.018182,0.0,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Sou...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.049383,0.0,0.037037,0.012346,0.000000,0.000000
4,Church and Wellesley,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.0,...,0.012048,0.012048,0.0,0.0,0.000000,0.0,0.012048,0.000000,0.012048,0.012048


In [141]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1          Bakery  0.04
2  Farmers Market  0.04
3        Beer Bar  0.04
4            Café  0.04


----CN Tower, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4           Airport  0.06


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.07
1  Chinese Restaurant  0.05
2                Café  0.05
3                Park  0.05
4         Pizza Place  0.05


----Chinatown, Grange Park, Kensington Market----
                           venue  freq
0                            Bar  0.07
1                           Café  0.06
2             Chinese Restaurant  0.05
3  Vegetarian / Vegan Restaurant  0.05
4                    Coffee Shop  0.05


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.07
1     Sushi Restaurant  0.05
2  

In [142]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [143]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Bakery,Cheese Shop,Cocktail Bar,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market,Creperie
1,"CN Tower, King and Spadina, Railway Lands, Sou...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Sculpture Garden,Boat or Ferry,Bar,Coffee Shop,Plane
2,"Cabbagetown, St. James Town",Coffee Shop,Italian Restaurant,Park,Chinese Restaurant,Restaurant,Pizza Place,Bakery,Pub,Café,Grocery Store
3,"Chinatown, Grange Park, Kensington Market",Bar,Café,Coffee Shop,Chinese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Grocery Store,Cocktail Bar
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Fast Food Restaurant,Gym,Hotel,Café,Gastropub
5,Commerce Court,Coffee Shop,Café,Hotel,Restaurant,Gym,Bakery,Italian Restaurant,Gastropub,Steakhouse,Seafood Restaurant
6,"Deer Park, Rathnelly, South Hill",Coffee Shop,Pub,Sushi Restaurant,Restaurant,Supermarket,Sports Bar,Liquor Store,Pizza Place,Fried Chicken Joint,American Restaurant
7,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Seafood Restaurant,Bar,Gastropub,American Restaurant,Deli / Bodega
8,Dovercourt Village,Pharmacy,Bakery,Grocery Store,Bar,Park,Bank,Gym / Fitness Center,Music Venue,Supermarket,Brewery
9,"Exhibition Place, Parkdale Village",Café,Coffee Shop,Performing Arts Venue,Breakfast Spot,Furniture / Home Store,Pet Store,Italian Restaurant,Intersection,Gym,Yoga Studio


In [144]:
toronto_onehot.shape

(1174, 206)

In [145]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Sou...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.049383,0.000000,0.037037,0.012346,0.000000,0.000000
4,Church and Wellesley,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,...,0.012048,0.012048,0.00000,0.00,0.000000,0.000000,0.012048,0.000000,0.012048,0.012048
5,Commerce Court,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
6,"Deer Park, Rathnelly, South Hill",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000
7,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.010000,0.000000,0.00000,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Dovercourt Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478


In [146]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1          Bakery  0.04
2  Farmers Market  0.04
3        Beer Bar  0.04
4            Café  0.04


----CN Tower, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4           Airport  0.06


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.07
1  Chinese Restaurant  0.05
2                Café  0.05
3                Park  0.05
4         Pizza Place  0.05


----Chinatown, Grange Park, Kensington Market----
                           venue  freq
0                            Bar  0.07
1                           Café  0.06
2             Chinese Restaurant  0.05
3  Vegetarian / Vegan Restaurant  0.05
4                    Coffee Shop  0.05


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.07
1     Sushi Restaurant  0.05
2  

In [147]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [148]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Bakery,Cheese Shop,Cocktail Bar,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market,Creperie
1,"CN Tower, King and Spadina, Railway Lands, Sou...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Sculpture Garden,Boat or Ferry,Bar,Coffee Shop,Plane
2,"Cabbagetown, St. James Town",Coffee Shop,Italian Restaurant,Park,Chinese Restaurant,Restaurant,Pizza Place,Bakery,Pub,Café,Grocery Store
3,"Chinatown, Grange Park, Kensington Market",Bar,Café,Coffee Shop,Chinese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Grocery Store,Cocktail Bar
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Fast Food Restaurant,Gym,Hotel,Café,Gastropub
5,Commerce Court,Coffee Shop,Café,Hotel,Restaurant,Gym,Bakery,Italian Restaurant,Gastropub,Steakhouse,Seafood Restaurant
6,"Deer Park, Rathnelly, South Hill",Coffee Shop,Pub,Sushi Restaurant,Restaurant,Supermarket,Sports Bar,Liquor Store,Pizza Place,Fried Chicken Joint,American Restaurant
7,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Seafood Restaurant,Bar,Gastropub,American Restaurant,Deli / Bodega
8,Dovercourt Village,Pharmacy,Bakery,Grocery Store,Bar,Park,Bank,Gym / Fitness Center,Music Venue,Supermarket,Brewery
9,"Exhibition Place, Parkdale Village",Café,Coffee Shop,Performing Arts Venue,Breakfast Spot,Furniture / Home Store,Pet Store,Italian Restaurant,Intersection,Gym,Yoga Studio


### Clustering

In [156]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [158]:
toronto_merged = df_comb_t

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Pub,Café,Bakery,Mexican Restaurant,Breakfast Spot,Theater,Restaurant,Chocolate Shop
2,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,0,Bar,Restaurant,Coffee Shop,Asian Restaurant,Pizza Place,Men's Store,Vietnamese Restaurant,Café,Yoga Studio,Brewery
3,M5S,Downtown Toronto,University of Toronto,43.662696,-79.400049,0,Café,Bakery,Japanese Restaurant,Bookstore,Bar,Restaurant,Sushi Restaurant,Sandwich Place,Italian Restaurant,Pub
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Café,Bakery,Cheese Shop,Cocktail Bar,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market,Creperie


In [159]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters